In [ ]:
import argparse
import os
import random
import sys
from pathlib import Path
from random import shuffle

import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import yaml
from cellpose import models
from matplotlib import pyplot as plt
from phenoscapes.cli import load_default_config, process_sample
from phenoscapes.feature_extraction import extract_features
from phenoscapes.sc import convert_to_h5ad, plot_summary
from phenoscapes.segmentation import generate_label_overlay, run_cellpose
from phenoscapes.utils import get_metadata, scale_image
from skimage import io
from skimage.color import label2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops
from tqdm import tqdm

In [ ]:
os.chdir("/Brain_ECM_4i_2_v2/")
dir_output = os.getcwd()
df_stains = pd.read_csv("stain_metadata.csv")
dir_bg_subtracted = Path(dir_output, "bg_subtracted")

In [ ]:
samples = [
    "R066_0",
    "R062_0",
    "R073_0",
    "R072_0",
    "R026_0",
    "R022_0",
    "R024_0",
    "R049_0",
    "R055_0",
    "R044_0",
    "R057_0",
    "R076_1",
    "R087_0",
    "R082_0",
    "R108_1",
    "R112_1",
    "R113_1",
    "R115_0",
]

In [ ]:
dir_images = dir_bg_subtracted
for sample in samples:
    # Create and shuffle color map and use to color compartements
    random.seed(10)
    dir_segmented_cytoplasma = Path(dir_output, "segmented_cytoplasma")
    dir_segmented_ec_niche = Path(dir_output, "segmented_ecm_niche")
    dir_segmented_nuclei = Path(dir_output, "segmented_cell_nuclei")
    dirs_segmented = [
        dir_segmented_cytoplasma,
        dir_segmented_ec_niche,
        dir_segmented_nuclei,
    ]
    img_label = io.imread(Path(dir_segmented_cytoplasma, sample + ".tif"))
    out_image = np.zeros(img_label.shape + (3,))
    mycmap = mcolors.LinearSegmentedColormap.from_list(
        "custom", ["white", "#525252"], N=20, gamma=1
    )
    greys = [
        matplotlib.colors.to_rgb(mcolors.rgb2hex(mycmap(i))) for i in range(mycmap.N)
    ]
    shuffle(greys)

    for dir_segmented, i in zip(dirs_segmented, range(len(dirs_segmented))):
        img_label = io.imread(Path(dir_segmented, sample + ".tif"))

        img_overlay = label2rgb(img_label, bg_label=0, colors=greys)
        out_image[:, :, i] = np.mean(img_overlay, axis=-1)
    out_image = (out_image * 255).astype("uint8")
    io.imsave(
        Path(f"/compartment_overlays/{sample}.png"),
        out_image,
        check_contrast=False,
    )